In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from collections import Counter

In [2]:
url = 'https://bestrestaurantsparis.com/fr/restaurant-paris/41-penthievre-restaurant-paris.html'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
soup.title.string

'Restaurant Au 41 Penthièvre, Paris 8ème | bestrestaurantsparis.com'

In [3]:
dishes = soup.find_all('div', class_='restaurant-menu-item')
dishes

[<div class="restaurant-menu-item">
 <div class="restaurant-menu-price"><span> 16,00 €</span></div>
 <div class="restaurant-menu-desc">Velouté de potiron du Domaine des Bagots en Sologne éclats de châtaignes, et dès de foie gras</div>
 </div>,
 <div class="restaurant-menu-item">
 <div class="restaurant-menu-price"><span> 18,00 €</span></div>
 <div class="restaurant-menu-desc">Oeuf de poule cuit mollet roulé à la mie de pain girolles de Corrèze en persillade légère</div>
 </div>,
 <div class="restaurant-menu-item">
 <div class="restaurant-menu-price"><span> 19,00 €</span></div>
 <div class="restaurant-menu-desc">Darphin de pomme de terre, saumon fumé "Label Rouge" fleurette aux perles d'Avruga</div>
 </div>,
 <div class="restaurant-menu-item">
 <div class="restaurant-menu-price"><span> 25,00 €</span></div>
 <div class="restaurant-menu-desc">Gelée de tourteaux à l'infusion d'herbes bavaroise de crustacés au fenouil et caviar de harengs</div>
 </div>,
 <div class="restaurant-menu-item">
 

In [4]:
# Extract the dish names and prices
data = []
for dish in dishes:
    price = dish.find('div', class_='restaurant-menu-price').get_text(strip=True)
    description = dish.find('div', class_='restaurant-menu-desc').get_text(strip=True)
    data.append([price, description])

# Create a DataFrame
df = pd.DataFrame(data, columns=['Price', 'Description'])
df


,Price,Description
0,"16,00 €",Velouté de potiron du Domaine des Bagots en So...
1,"18,00 €",Oeuf de poule cuit mollet roulé à la mie de pa...
2,"19,00 €","Darphin de pomme de terre, saumon fumé ""Label ..."
3,"25,00 €",Gelée de tourteaux à l'infusion d'herbes bavar...
4,"19,00 €",Escalope de foie gras de canard poêlé du Sud-O...
5,"39,00 €",Ravioles de langoustines Bretonnes bisque à l'...
6,"41,00 €",Suprême de bar en croûte d'amandes ravigote au...
7,"37,00 €",Magret de canard du Sud-Ouest caramélisé aux d...
8,"49,00 €","Filet de boeuf de Salers cuit au poêlon, sauce..."
9,"46,00 €","Noix de ris de veau laqué au vin rouge, pois g..."


In [5]:
data

[['16,00 €',
  'Velouté de potiron du Domaine des Bagots en Sologne éclats de châtaignes, et dès de foie gras'],
 ['18,00 €',
  'Oeuf de poule cuit mollet roulé à la mie de pain girolles de Corrèze en persillade légère'],
 ['19,00 €',
  'Darphin de pomme de terre, saumon fumé "Label Rouge" fleurette aux perles d\'Avruga'],
 ['25,00 €',
  "Gelée de tourteaux à l'infusion d'herbes bavaroise de crustacés au fenouil et caviar de harengs"],
 ['19,00 €',
  'Escalope de foie gras de canard poêlé du Sud-Ouest jus court aux mûres et groseilles'],
 ['39,00 €',
  "Ravioles de langoustines Bretonnes bisque à l'Armoricaine et estragon"],
 ['41,00 €',
  "Suprême de bar en croûte d'amandes ravigote aux câpres et citron, tétragones en salade"],
 ['37,00 €',
  "Magret de canard du Sud-Ouest caramélisé aux dragées coings rôtis au miel d'acacia"],
 ['49,00 €',
  "Filet de boeuf de Salers cuit au poêlon, sauce Béarnaise, jeunes légumes du moment à l'huile d'olive"],
 ['46,00 €',
  'Noix de ris de veau laq

In [6]:
data[1]

['18,00 €',
 'Oeuf de poule cuit mollet roulé à la mie de pain girolles de Corrèze en persillade légère']

### Part 2: Meat database

In [13]:
url_base = "https://www.marmiton.org/recettes/index/categorie/"
categories = ['viande', 'poisson', 'oeufs', 'fruits-de-mer', 'plat-vegetarien', 'plats-au-fromage']

# Create a dictionary to store recipe titles and links for each category
all_recipes_by_category = {}

# Iterate through categories
for category in categories:
    url_category = url_base + category + '/'
    
    # Create lists to store recipe titles and links for the current category
    recipe_titles = []
    recipe_links = []
    
    # Iterate through pages from 1 to 200
    for i in range(1, 201):
        url_page = url_category + str(i)
        response = requests.get(url_page)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all anchor tags with the class 'recipe-card-link'
        recipe_cards = soup.find_all('a', class_='recipe-card-link')

        # Check if any recipe cards are found
        if recipe_cards:
            for card in recipe_cards:
                # Within each anchor tag, find the h4 element with the class 'recipe-card__title'
                title_element = card.find('h4', class_='recipe-card__title')
                
                # Find the link (href) of the recipe
                recipe_link = card['href']
                response1 = requests.get(recipe_link)
                recipe = BeautifulSoup(response1.text, 'html.parser')

                # Check if the title element is found before adding its text to the list
                if title_element:
                    recipe_titles.append(title_element.text.strip())
        else:
            # If no recipe cards are found, break out of the inner loop
            break

    # Add the lists of recipe titles and links to the dictionary with the category as the key
    all_recipes_by_category[category] = {'titles': recipe_titles, 'links': recipe_links}


# Print the dictionary containing recipe titles and links for each category
# print(all_recipes_by_category)

KeyboardInterrupt: 

['Blanquette de veau : recette traditionnelle',
 'Poulet au four simple et savoureux',
 'Boeuf bourguignon : la vraie recette',
 'Rôti de boeuf au four tout simple',
 'Filet mignon de porc à la moutarde',
 'Cuisses de poulet au four',
 'Rôti de porc tout simple',
 "Gigot d'agneau : la recette traditionnelle",
 'Paupiettes de veau',
 'Osso bucco de veau',
 'Falafel (croquettes de pois chiches)',
 'Poulet basquaise',
 'Rôti de veau au four',
 "Souris d'agneau au miel et thym",
 'Sauté de porc à la crème de moutarde',
 'La côte de bœuf au four facile et cuite à la perfection',
 'Poulet rôti et ses pommes de terre',
 "Rosbeef au four à l'ail",
 'Rôti de porc : recette savoureuse',
 "Filet mignon de porc rôti à la moutarde à l'ancienne",
 "Epaule d'agneau confite facile",
 'Lentilles corail au curry',
 'poulet au coco et curry',
 'Poulet curry et oignons facile',
 "Rouelle de porc à l'ancienne",
 'Joue de porc à la bière',
 'Colombo de poulet',
 'Joue de boeuf à la bourguignonne',
 'Andouil

In [16]:
#### Recipe + Ingredients

import requests
from bs4 import BeautifulSoup

url_base = "https://www.marmiton.org/recettes/index/categorie/"
categories = ['viande', 'poisson', 'oeufs', 'fruits-de-mer', 'plat-vegetarien', 'plats-au-fromage']
#categories = ['viande','plat-vegetarien']
# Create a dictionary to store recipe titles, links, and ingredient names for each category
all_recipes_by_category = {}

# Iterate through categories
for category in categories:
    url_category = url_base + category + '/'
    
    # Create lists to store recipe titles, links, and ingredient names for the current category
    recipe_titles = []
    #recipe_links = []
    ingredient_names_list = []  # New list to store ingredient names
    
    # Iterate through pages from 1 to 200
    for i in range(1, 6):
        url_page = url_category + str(i)
        response = requests.get(url_page)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all anchor tags with the class 'recipe-card-link'
        recipe_cards = soup.find_all('a', class_='recipe-card-link')

        # Check if any recipe cards are found
        if recipe_cards:
            for card in recipe_cards:
                # Within each anchor tag, find the h4 element with the class 'recipe-card__title'
                title_element = card.find('h4', class_='recipe-card__title')
                
                # Find the link (href) of the recipe
                recipe_link = card['href']
                response1 = requests.get(recipe_link)
                recipe = BeautifulSoup(response1.text, 'html.parser')

                # Check if the title element is found before adding its text to the list
                if title_element:
                    recipe_titles.append(title_element.text.strip())
                    #recipe_links.append(recipe_link)
                    
                    # Extract ingredient names from card-ingredient elements
                    card_ingredient_elements = recipe.find_all('div', class_='card-ingredient')
                    ingredient_names = [element['data-name'] for element in card_ingredient_elements]
                    ingredient_names_list.append(ingredient_names)
        else:
            # If no recipe cards are found, break out of the inner loop
            break

    # Add the lists of recipe titles, links, and ingredient names to the dictionary with the category as the key
    all_recipes_by_category[category] = {'titles': recipe_titles, 
                                         #'links': recipe_links, 
                                         'ingredients': ingredient_names_list}


# Print the dictionary containing recipe titles, links, and ingredient names for each category
print(all_recipes_by_category)


{'viande': {'titles': ['Blanquette de veau : recette traditionnelle', 'Poulet au four simple et savoureux', 'Boeuf bourguignon : la vraie recette', 'Rôti de boeuf au four tout simple', 'Filet mignon de porc à la moutarde', 'Cuisses de poulet au four', 'Rôti de porc tout simple', "Gigot d'agneau : la recette traditionnelle", 'Paupiettes de veau', 'Osso bucco de veau', 'Falafel (croquettes de pois chiches)', 'Poulet basquaise', 'Rôti de veau au four', "Souris d'agneau au miel et thym", 'Sauté de porc à la crème de moutarde', 'La côte de bœuf au four facile et cuite à la perfection', 'Poulet rôti et ses pommes de terre', "Rosbeef au four à l'ail", 'Rôti de porc : recette savoureuse', "Filet mignon de porc rôti à la moutarde à l'ancienne", "Epaule d'agneau confite facile", 'Lentilles corail au curry', 'poulet au coco et curry', 'Poulet curry et oignons facile', "Rouelle de porc à l'ancienne", 'Joue de porc à la bière', 'Colombo de poulet', 'Joue de boeuf à la bourguignonne', 'Andouillette 

In [34]:
card_ingredient_elements

[<div class="card-ingredient" data-brandid="163" data-brandname="Amazon" data-name="sel" data-visibleonfirstdisplay="true">
 <span class="card-ingredient-link af-to-obfuscate-58899" data-encoded-link="aHR0cHM6Ly9mbGl6Lmx5L3FXVzVfdw==" target="_blank"> <div class="card-ingredient-image">
 <img alt="" class="lazyload item__icon" data-src="https://assets.afcdn.com/recipe/20170607/67687_origin.jpg" height="150px" src="https://static.afcdn.com/relmrtn/lazyload.png" width="150px"> </img></div>
 </span> <div class="card-ingredient-content">
 <div class="card-ingredient-checkbox">
 <input class="checkbox" id="check990" type="checkbox">
 <label for="check990"></label>
 </input></div>
 <span class="card-ingredient-link af-to-obfuscate-58899" data-encoded-link="aHR0cHM6Ly9mbGl6Lmx5L3FXVzVfdw==" target="_blank"> <span class="card-ingredient-title">
 <span class="card-ingredient-quantity" data-ingredientquantity="0">
 <span class="count"></span>
 <span class="unit" data-unitplural="" data-unitsingu

## NB: Si besoin de chercher des categories

In [9]:
url = "https://www.marmiton.org/recettes/index/categorie/plat-principal/"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find the unordered list with the class 'mrtn-tags-list'
tag_list = soup.find('ul', class_='mrtn-tags-list')

# Extract all the category names from the list items
categories = [li.a.text for li in tag_list.find_all('li', class_='mrtn-tag')]

# Print the list of category names
print(categories)


['viande', 'poisson', 'fruits de mer', 'plat unique', 'œufs', 'plat végétarien', 'pâtes, riz, semoule', 'plats au fromage']


In [10]:
all_words = ' '.join(all_recipe_titles_by_category['viande'])
word_counts = Counter(all_words.split())
df=pd.DataFrame(list(word_counts.items()),columns=['Word','Occurence'])
rm=["de","le","la","les","et","au","aux","à","la",""]
df=(df.sort_values(by='Occurence',ascending=False)).reset_index(drop=True)
df[:100]

,Word,Occurence
0,de,4201
1,au,2211
2,et,1652
3,aux,1507
4,à,1309
...,...,...
95,fromage,44
96,noix,43
97,jambon,43
98,fruits,42


In [11]:
def process_category(data, category_name):
    # Convert all words to lowercase
    data_lower = [word.lower() for word in data]

    # Flatten the list of strings into a single string
    all_words = ' '.join(data_lower)

    # Split the string into words and count occurrences
    word_counts = Counter(all_words.split())

    # Create a DataFrame from the Counter dictionary
    df = pd.DataFrame(list(word_counts.items()), columns=['Word', f'Occurrence_{category_name}'])

    # Remove specified words
    stop_words = ["de", "la", "et", "au", "aux", "à", "la", "en", "des"]
    df = df[~df['Word'].isin(stop_words)]

    # Sort the DataFrame by Occurrence in descending order
    df = df.sort_values(by=f'Occurrence_{category_name}', ascending=False)

    # Reinitialize the index
    df = df.reset_index(drop=True)

    return df

# Create and process DataFrames for each category
df_viande = process_category(all_recipe_titles_by_category['viande'], 'viande')
df_poisson = process_category(all_recipe_titles_by_category['poisson'], 'poisson')
df_vegetarien = process_category(all_recipe_titles_by_category['plat-vegetarien'], 'vegetarien')

# Find common words between the three DataFrames
common_words = set(df_viande['Word']) & set(df_poisson['Word']) & set(df_vegetarien['Word'])

# Remove common words from each DataFrame
df_viande = df_viande[~df_viande['Word'].isin(common_words)]
df_poisson = df_poisson[~df_poisson['Word'].isin(common_words)]
df_vegetarien = df_vegetarien[~df_vegetarien['Word'].isin(common_words)]

# Display the processed DataFrames
print("viande DataFrame:")
print(df_viande)

print("\npoisson DataFrame:")
print(df_poisson)

print("\nvegetarien DataFrame:")
print(df_vegetarien)

viande DataFrame:
                   Word  Occurrence_viande
1                  porc                710
2                  foie                521
4                  veau                440
6                  gras                417
9                mignon                318
...                 ...                ...
2915  endive-poire-foie                  1
2916           tiramisu                  1
2917    (chèvres-curry)                  1
2918       mini-légumes                  1
2919             tomaté                  1

[2742 rows x 2 columns]

poisson DataFrame:
           Word  Occurrence_poisson
1        filets                 166
3       poisson                 142
5     cabillaud                 108
12         pavé                  43
14        pavés                  41
..          ...                 ...
825   d'amandes                   1
826      croute                   1
827   d'épinard                   1
828  italiennes                   1
829        poil          

In [12]:
common_words

{'&',
 '(facile)',
 '-',
 '2',
 '3',
 ':',
 'a',
 'avec',
 'bacon',
 'banane',
 'basilic',
 'blanc',
 'blé',
 'brocolis',
 'burger',
 'béchamel',
 'carotte',
 'carottes',
 'champignons',
 'chef',
 'chorizo',
 'chou',
 'choux',
 'chèvre',
 'coco',
 'coeur',
 'colin',
 'companion',
 'comté',
 'concombres',
 'confit',
 'cookeo',
 'cooking',
 'courgettes',
 'courgettes,',
 'crème',
 'crémeux',
 'cuisine',
 'cuite',
 'cumin',
 'curry',
 'curry,',
 'céréales',
 "d'automne",
 "d'herbes",
 "d'été",
 'deux',
 'douce',
 'douces',
 'du',
 'et...',
 'facile',
 'farcie',
 'farcies',
 'farcis',
 'façon',
 'feta',
 'figues',
 'filet',
 'flamande',
 'fleur',
 'fondant',
 'forestière',
 'four',
 'frais',
 'fraîche',
 'fricassée',
 'fromage',
 'fruits',
 'fumée',
 'gingembre',
 'gnocchi',
 'gorgonzola',
 'graines',
 'gratin',
 'gratinés',
 'grillé',
 'haricots',
 'herbes',
 'jambon',
 'julienne',
 "l'ail",
 "l'huile",
 "l'italienne",
 'lait',
 'lard',
 'lardons',
 'laurent',
 'le',
 'lentilles',
 'les',

In [13]:
def process_category(data, category_name):
    # Convert all words to lowercase
    data_lower = [word.lower() for word in data]

    # Flatten the list of strings into a single string
    all_words = ' '.join(data_lower)

    # Split the string into words and count occurrences
    word_counts = Counter(all_words.split())

    # Create a DataFrame from the Counter dictionary
    df = pd.DataFrame(list(word_counts.items()), columns=['Word', f'Occurrence_{category_name}'])

    # Remove specified words
    stop_words = ["de", "la", "et", "au", "aux", "à", "la", "en", "des"]
    df = df[~df['Word'].isin(stop_words)]

    # Sort the DataFrame by Occurrence in descending order
    df = df.sort_values(by=f'Occurrence_{category_name}', ascending=False)

    # Reinitialize the index
    df = df.reset_index(drop=True)

    return df

# Create and process DataFrames for each category
df_viande = process_category(all_recipe_titles_by_category['viande'], 'viande')
df_poisson = process_category(all_recipe_titles_by_category['poisson'], 'poisson')
df_vegetarien = process_category(all_recipe_titles_by_category['plat-vegetarien'], 'vegetarien')

# Find common words between the three DataFrames
common_words = set(df_viande['Word']) & set(df_poisson['Word']) & set(df_vegetarien['Word'])

# Keep common words in the DataFrame with the highest occurrence
for word in common_words:
    occurrences_viande = df_viande.loc[df_viande['Word'] == word, f'Occurrence_viande'].values
    occurrences_poisson = df_poisson.loc[df_poisson['Word'] == word, f'Occurrence_poisson'].values
    occurrences_vegetarien = df_vegetarien.loc[df_vegetarien['Word'] == word, f'Occurrence_vegetarien'].values

    max_occurrence = max(occurrences_viande, occurrences_poisson, occurrences_vegetarien)

    if occurrences_viande and occurrences_viande[0] == max_occurrence:
        df_poisson = df_poisson[df_poisson['Word'] != word]
        df_vegetarien = df_vegetarien[df_vegetarien['Word'] != word]
    elif occurrences_poisson and occurrences_poisson[0] == max_occurrence:
        df_viande = df_viande[df_viande['Word'] != word]
        df_vegetarien = df_vegetarien[df_vegetarien['Word'] != word]
    elif occurrences_vegetarien and occurrences_vegetarien[0] == max_occurrence:
        df_viande = df_viande[df_viande['Word'] != word]
        df_poisson = df_poisson[df_poisson['Word'] != word]

# Display the processed DataFrames
print("viande DataFrame:")
print(df_viande)

print("\npoisson DataFrame:")
print(df_poisson)

print("\nvegetarien DataFrame:")
print(df_vegetarien)

viande DataFrame:
                   Word  Occurrence_viande
0                poulet               1275
1                  porc                710
2                  foie                521
3                 curry                464
4                  veau                440
...                 ...                ...
2915  endive-poire-foie                  1
2916           tiramisu                  1
2917    (chèvres-curry)                  1
2918       mini-légumes                  1
2919             tomaté                  1

[2901 rows x 2 columns]

poisson DataFrame:
           Word  Occurrence_poisson
0        saumon                 274
1        filets                 166
2     papillote                 155
3       poisson                 142
5     cabillaud                 108
..          ...                 ...
825   d'amandes                   1
826      croute                   1
827   d'épinard                   1
828  italiennes                   1
829        poil          

In [14]:
from collections import Counter
import pandas as pd
import string

def process_category(data, category_name):
    # Convert all words to lowercase and remove punctuation
    translator = str.maketrans("", "", string.punctuation)
    data_lower = [word.translate(translator).lower() for word in data]

    # Flatten the list of strings into a single string
    all_words = ' '.join(data_lower)

    # Split the string into words and count occurrences
    word_counts = Counter(all_words.split())

    # Create a DataFrame from the Counter dictionary
    df = pd.DataFrame(list(word_counts.items()), columns=['Word', f'Occurrence_{category_name}'])

    # Remove specified words
    stop_words = ["de", "la", "et", "au", "aux", "à", "la", "en", "des"]
    df = df[~df['Word'].isin(stop_words)]

    # Remove words that occur only once
    df = df[df[f'Occurrence_{category_name}'] > 1]

    # Sort the DataFrame by Occurrence in descending order
    df = df.sort_values(by=f'Occurrence_{category_name}', ascending=False)

    # Reinitialize the index
    df = df.reset_index(drop=True)

    return df

# Create and process DataFrames for each category
df_viande = process_category(all_recipe_titles_by_category['viande'], 'viande')
df_poisson = process_category(all_recipe_titles_by_category['poisson'], 'poisson')
df_vegetarien = process_category(all_recipe_titles_by_category['plat-vegetarien'], 'vegetarien')

# Find common words between the three DataFrames
common_words = set(df_viande['Word']) & set(df_poisson['Word']) & set(df_vegetarien['Word'])

# Keep common words in the DataFrame with the highest occurrence
for word in common_words:
    occurrences_viande = df_viande.loc[df_viande['Word'] == word, f'Occurrence_viande'].values
    occurrences_poisson = df_poisson.loc[df_poisson['Word'] == word, f'Occurrence_poisson'].values
    occurrences_vegetarien = df_vegetarien.loc[df_vegetarien['Word'] == word, f'Occurrence_vegetarien'].values

    max_occurrence = max(occurrences_viande, occurrences_poisson, occurrences_vegetarien)

    if occurrences_viande and occurrences_viande[0] == max_occurrence:
        df_poisson = df_poisson[df_poisson['Word'] != word]
        df_vegetarien = df_vegetarien[df_vegetarien['Word'] != word]
    elif occurrences_poisson and occurrences_poisson[0] == max_occurrence:
        df_viande = df_viande[df_viande['Word'] != word]
        df_vegetarien = df_vegetarien[df_vegetarien['Word'] != word]
    elif occurrences_vegetarien and occurrences_vegetarien[0] == max_occurrence:
        df_viande = df_viande[df_viande['Word'] != word]
        df_poisson = df_poisson[df_poisson['Word'] != word]

# Display the processed DataFrames
print("viande DataFrame:")
print(df_viande)

print("\npoisson DataFrame:")
print(df_poisson)

print("\nvegetarien DataFrame:")
print(df_vegetarien)


viande DataFrame:
                 Word  Occurrence_viande
0              poulet               1306
1                porc                718
2                foie                525
3               curry                483
4                veau                444
...               ...                ...
1170      sucréesalée                  2
1171  méditerranéenne                  2
1172             saté                  2
1173           truffé                  2
1174              mam                  2

[1167 rows x 2 columns]

poisson DataFrame:
          Word  Occurrence_poisson
0       saumon                 279
1       filets                 166
2    papillote                 158
3      poisson                 146
4        filet                 111
..         ...                 ...
310     barbet                   2
311   estragon                   2
312   pistache                   2
313     mangue                   2
314   pruneaux                   2

[240 rows x 2 columns]

